In [1]:
import requests
import os
import hashlib
import io
import time
import pandas as pd

In [2]:
# Download the db
"https://onedrive.live.com/download?cid=FD520DDC6BE92730&resid=FD520DDC6BE92730%21616&authkey=AEeP_4E1uh-vyDE"

'https://onedrive.live.com/download?cid=FD520DDC6BE92730&resid=FD520DDC6BE92730%21616&authkey=AEeP_4E1uh-vyDE'

In [2]:
# Connect
DB_FILENAME = 'NYC-311-2M.db'
import sqlite3 as db
disk_engine = db.connect(DB_FILENAME)

In [3]:
# Perform SQL query through the disk_engine connection.
# The return value is a pandas data frame.
df = pd.read_sql_query ('SELECT * FROM data', disk_engine)

# Dump the first few rows
df.head()

,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2015-09-15 02:14:04.000000,None,NYPD,Illegal Parking,Blocked Hydrant,None
1,2,2015-09-15 02:12:49.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
2,3,2015-09-15 02:11:19.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
3,4,2015-09-15 02:09:46.000000,None,NYPD,Noise - Commercial,Loud Talking,BRONX
4,5,2015-09-15 02:08:01.000000,2015-09-15 02:08:18.000000,DHS,Homeless Person Assistance,Status Call,NEW YORK


In [4]:
query = '''
  SELECT *
    FROM data
    LIMIT 5
'''
start_time = time.time ()
df = pd.read_sql_query (query, disk_engine)
elapsed_time = time.time () - start_time
print ("==> LIMIT version took %g seconds." % elapsed_time)

df

==> LIMIT version took 0.0030005 seconds.


,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2015-09-15 02:14:04.000000,None,NYPD,Illegal Parking,Blocked Hydrant,None
1,2,2015-09-15 02:12:49.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
2,3,2015-09-15 02:11:19.000000,None,NYPD,Noise - Street/Sidewalk,Loud Talking,NEW YORK
3,4,2015-09-15 02:09:46.000000,None,NYPD,Noise - Commercial,Loud Talking,BRONX
4,5,2015-09-15 02:08:01.000000,2015-09-15 02:08:18.000000,DHS,Homeless Person Assistance,Status Call,NEW YORK


__Grouping Information: GROUP BY operator__. The GROUP BY operator lets you group information using a particular column or multiple columns of the table. The output generated is more of a pivot table.



In [5]:
query = '''
    SELECT ComplaintType, Descriptor, Agency
    FROM data
    GROUP BY ComplaintType
    ORDER BY ComplaintType
    LIMIT 10
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,ComplaintType,Descriptor,Agency
0,AGENCY,HOUSING QUALITY STANDARDS,HPD
1,APPLIANCE,REFRIGERATOR,HPD
2,Adopt-A-Basket,10A Adopt-A-Basket,DSNY
3,Agency Issues,Call Center Compliment,3-1-1
4,Air Quality,"Air: Odor/Fumes, Vehicle Idling (AD3)",DEP


__Set membership: IN operator__. Another common idiom is to ask for rows whose attributes fall within a set, for which you can use the IN operator.

In [6]:
query = '''
    SELECT ComplaintType, Descriptor, Agency
    FROM data
    WHERE Agency IN ("NYPD", "DOB")
    LIMIT 10
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,ComplaintType,Descriptor,Agency
0,Illegal Parking,Blocked Hydrant,NYPD
1,Noise - Street/Sidewalk,Loud Talking,NYPD
2,Noise - Street/Sidewalk,Loud Talking,NYPD
3,Noise - Commercial,Loud Talking,NYPD
4,Blocked Driveway,Partial Access,NYPD


__Finding unique values: DISTINCT qualifier__. Yet another common idiom is to ask for the unique values of some attribute, for which you can use the DISTINCT qualifier.

In [7]:
query = 'SELECT DISTINCT City FROM data'
df = pd.read_sql_query(query, disk_engine)

df.head()

,City
0,None
1,NEW YORK
2,BRONX
3,STATEN ISLAND
4,ELMHURST


__Renaming columns__: AS __operator__. Sometimes you might want to rename a result column. For instance, the following query counts the number of complaints by "Agency," using the COUNT(*) function and GROUP BY clause, which we discussed in an earlier lab. If you wish to refer to the counts column of the resulting data frame, you can give it a more "friendly" name using the  AS operator.

In [8]:
query = '''
    SELECT Agency, COUNT(*) AS NumComplaints
    FROM data
    GROUP BY Agency
'''

df = pd.read_sql_query(query, disk_engine)
df.head()

,Agency,NumComplaints
0,3-1-1,1289
1,ACS,3
2,AJC,6
3,CAU,1
4,CCRB,1


In [9]:
disk_engine.close()